In [2]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import locale
import os

### Load in nnhe array Data

In [3]:
meta = []

with open('data-he.csv') as f:
    for line in f.readlines():
        meta.append(eval(line))

meta = np.array(meta)
np.save('nnhe-meta-simple-minturn-1', meta)

### Load in player hand data

In [4]:
player_hands = []

with open('data-ph.csv') as f:
    for line in f.readlines():
        player_hands.append(eval(line))

player_hands = np.array([arr.flatten() for arr in np.array(player_hands)])
np.save('nnhe-player-hands-simple-minturn-1', player_hands)

### From np file

In [7]:
meta = np.load('nnhe-meta-simple-minturn-1.npy')
player_hands = np.load('nnhe-player-hands-simple-minturn-1.npy')

### split train and test data

In [8]:
i = int(len(meta) / 2)
j = int(len(meta) * 3 / 4)

trainX = meta[:i]
testX = meta[i:j]
endX = meta[j:]

trainY = player_hands[:i]
testY = player_hands[i:j]
endY = player_hands[j:]

### Create model

In [9]:
model = Sequential()
model.add(Dense(100, input_dim=trainX.shape[1], activation="relu"))
model.add(Dense(85, activation="sigmoid"))
model.add(Dense(60, activation="sigmoid"))
model.add(Dense(52, activation="sigmoid"))

opt = Adam(
       learning_rate = 0.001,     # 0.001
       beta_1=0.9,                # 0.9
       beta_2=0.999,              # 0.999
       epsilon=0.0001,            # 1e-7
       amsgrad=False,             # False
       name="Adam"
      )

model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['accuracy', 'mean_squared_error', 'categorical_crossentropy'])

### Fit

In [10]:
model.fit(
        trainX, trainY,
        validation_data=(testX, testY),
        epochs=200, batch_size=50
)

Epoch 1/200
25540/25540 [==============================] - 26s 1ms/step - loss: 0.1472 - accuracy: 0.0388 - mean_squared_error: 0.1472 - categorical_crossentropy: 38.3808 - val_loss: 0.1434 - val_accuracy: 0.0464 - val_mean_squared_error: 0.1434 - val_categorical_crossentropy: 37.8919
Epoch 2/200
25540/25540 [==============================] - 26s 1ms/step - loss: 0.1414 - accuracy: 0.0445 - mean_squared_error: 0.1414 - categorical_crossentropy: 37.5940 - val_loss: 0.1404 - val_accuracy: 0.0548 - val_mean_squared_error: 0.1404 - val_categorical_crossentropy: 37.3909
Epoch 3/200
25540/25540 [==============================] - 25s 994us/step - loss: 0.1393 - accuracy: 0.0425 - mean_squared_error: 0.1393 - categorical_crossentropy: 37.2235 - val_loss: 0.1391 - val_accuracy: 0.0298 - val_mean_squared_error: 0.1391 - val_categorical_crossentropy: 37.1573
Epoch 4/200
25540/25540 [==============================] - 25s 994us/step - loss: 0.1385 - accuracy: 0.0462 - mean_squared_error: 0.1385 - c

In [11]:
model.save('nnhe-simple-minturn-1.h5')

# End - Testing

In [12]:
def largest(arr, n):
    new_arr = [*arr]
    for i in range(n):
        new_arr[new_arr.index(max(new_arr))] = -1
    return [1 if new_arr[i] == -1 else 0 for i in range(52)]

In [13]:
t = 6

X = endX
Y = endY
P = model.predict(np.array([X[t]]))[0]


np.set_printoptions(suppress=True)

print("test case:", t)
# print("stage:", sum(list(X[t][i]).count(-1) + list(X[t][i]).count(1) for i in range(4)))
print()
print("meta:")
print(X[t])
print()
print("Prediction:")
print(P)
print()
print("  pred:"," ".join('{:.2f}'.format(round(e,2)) for e in P))
print("actual:", " ".join(f"{e}   "[:4] for e in Y[t]))
print("\n", "-" * 150, "\n")
print("rounded:", *[int(round(p)) for p in P])
print(" actual:", *[e for e in Y[t]])
print("largest:", *largest(P, 15))
print()
print("mean squared diff:", sum((P - Y[t]) ** 2) / 52)

test case: 6

meta:
[ 0.23809524  0.23809524  0.23809524  0.23809524  0.23809524  0.23809524
  0.          0.23809524  0.23809524  0.          0.23809524  0.23809524
  0.23809524  0.          0.23809524  0.23809524  0.23809524  0.23809524
  0.23809524  0.23809524  0.23809524  0.23809524  0.23809524  0.23809524
  0.23809524  0.23809524  0.23809524  0.          0.          0.23809524
  0.23809524  0.23809524  0.23809524  0.23809524  0.23809524  0.23809524
  0.          0.23809524  0.23809524  0.          0.23809524  0.23809524
  0.23809524  0.          0.          0.23809524  0.23809524  0.23809524
  0.          0.23809524  0.23809524  0.23809524  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.  